<a href="https://colab.research.google.com/github/xaviorlewis/arete.gallery/blob/main/get_global_lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Scrape genius.com and save the lyrics from each song to spotify feature dataframe

In [ ]:
pip install lyricsgenius

     |████████████████████████████████| 59 kB 3.3 MB/s 


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import lyricsgenius as lg
import pandas as pd
import json
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
api_info = {
    "ClientID" : "YNOKNqvbnvzE-Ygide-kpRPm0rZrlrrKDdGQ-CJLsX2S_7pV6hluRT88DPsqmc-P",
    "ClientSecret": "Qs8YPv3ulCd0pjpYSYUXRCjHeBYrDKudO0b3698rk7ds5bxRbCj37cqV6n1doyO7orVGwbzdre3orIYCI4jnAQ",
    "Token" : "SvM4VfnkL9dADv_XrwwJZyOSPMtKumh0NtOFLVXwcheYrOwzkeM5SHXtSxhjqitO"
  }
genius = lg.Genius(api_info["Token"],  # Client access token from Genius Client API page
                             skip_non_songs=True,
                             remove_section_headers=True)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/QRML/projects/final/data/feature_df.csv')

In [ ]:
#a function to clean up song titles gotten from Genius.com 
def clean_up(song_title):

    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
    
    elif "(with" in song_title:
        before_ft_pattern = re.compile(".*(?=\((with))")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
    
    elif "(feat" in song_title:
        before_ft_pattern = re.compile(".*(?=\((feat))")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()

    
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
    
    return clean_song_title

In [ ]:
#get all track_names
songs = []
for index, row in df.iterrows():
    song = row['track_name']
    songs.append(song)


In [ ]:
#clean up song titles
spiffy= []
for song in songs:
  cleaned = clean_up(song)
  spiffy.append(cleaned)

In [ ]:
#add cleaned titles to the dataframe
df["Cleaned_Titles"] = spiffy
df

,Unnamed: 0,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Cleaned_Titles
0,0,Bad Bunny,X 100PRE,NI BIEN NI MAL,278kSqsZIiYp8p3QjYAqa8,0.826,0.449,5,-8.330,0,0.0674,0.000018,0.2730,0.337,128.028,236216,4,NI BIEN NI MAL
1,1,Bad Bunny,X 100PRE,200 Mph,5jGUJXnELDaDTgiRjD9lsg,0.850,0.500,8,-6.708,1,0.0665,0.000384,0.0947,0.816,160.020,170509,4,200 Mph
2,2,Bad Bunny,X 100PRE,Cuando Perriabas,5mj8WVFcKdGA8p9HWGTSLc,0.787,0.705,0,-7.582,1,0.0695,0.000001,0.1080,0.499,81.998,188654,4,Cuando Perriabas
3,3,Bad Bunny,X 100PRE,La Romana,1khmgu0pveJbkbpbkyvcQv,0.655,0.725,0,-5.497,1,0.1880,0.002640,0.0611,0.326,125.800,300579,4,La Romana
4,4,Bad Bunny,X 100PRE,Como Antes,69ZaPBHhRMRDjRpW1ivnOU,0.767,0.379,0,-10.348,1,0.0385,0.000145,0.2170,0.252,103.956,230578,4,Como Antes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,372,BTS,Wings,BTS Cypher 4,1QWxddJmOhQb1vDdyTipMR,0.817,0.797,1,-4.395,0,0.0840,0.000008,0.0551,0.362,119.977,294463,4,BTS Cypher 4
373,373,BTS,Wings,Am I Wrong,1ORnZ5RmTKhVxPrjORKxAE,0.761,0.834,6,-4.992,1,0.0494,0.000010,0.0868,0.680,107.988,213082,4,Am I Wrong
374,374,BTS,Wings,21st Century Girl,6RioKpKPuG1vuyU0wnlAZx,0.716,0.906,2,-3.539,1,0.0717,0.000000,0.0547,0.762,101.028,192597,4,21st Century Girl
375,375,BTS,Wings,2! 3!,0ma7lbs8BsVIv2kSaqOsDI,0.439,0.928,7,-4.096,1,0.2330,0.000000,0.1000,0.599,169.928,272537,4,2! 3!


In [ ]:
#get lyrics for songs

song_lyrics = {}

for songTitle in spiffy:
  song = genius.search_song(songTitle)
  if song is None:
    continue
  else:
    if songTitle not in song_lyrics.keys():
      song_lyrics[songTitle]=song.lyrics



In [ ]:
#write text files containing lyrics of all songs

with open(f'/content/drive/MyDrive/QRML/projects/final/data/messy_global/messy_lyrics.txt', 'w') as writefile:
  writefile.write(str(song_lyrics))

In [ ]:

song_df = song_df.reset_index()

In [ ]:
#create a dataframe with all lyrics

df_lyrics = pd.DataFrame.from_dict(song_lyrics, orient='index')

In [ ]:
df_lyrics = df_lyrics.reset_index()
df_lyrics

,level_0,index,0
0,0,NI BIEN NI MAL,Yeh-yeh\nYeh-yeh\nYeh-yeh\n\nSin ti no me va b...
1,1,200 Mph,¡Ju-Ju!\nYeh-yeah-yeah\nYeah-yeah-yeah-yeah\n\...
2,2,Cuando Perriabas,Yeah-yeah-yeah\nYeah\nYo te llevo pa' Coachell...
3,3,La Romana,"(Yeh, yeh, yeh)\n\nPásame la hookah, ¡eh!\nQue..."
4,4,Como Antes,"Yeh-yeh-yeh-yeh, ey\n\nAyer me acordé de la pr..."
...,...,...,...
369,369,BTS Cypher 4,"이름, 이름 sorry bae\n발음, 발음 sorry bae\n딕션, 딕션, 딕션..."
370,370,Am I Wrong,"Ooh, ooh\nOoh, ooh\n\nAm I wrong for thinkin' ..."
371,371,21st Century Girl,Give me an inch\nI promise I'll take a mile\nD...
372,372,2! 3!,"Yeah, yeah, yeah, yeah, yeah\nYeah, yeah, yeah..."


In [ ]:
#make sure our column names are correct
df_lyrics = df_lyrics.rename(columns={"index":"Cleaned_Titles"})

In [ ]:
#merge our song dataframe with the lyrics dataframe, using our Cleaned_Titles column as a map to ensure the two dataframes are lined up correctly
df_data = pd.merge(left=song_df, right=df_lyrics, how='left', left_on=['Cleaned_Titles'],
         right_on = ['Cleaned_Titles'])
df_data

,Cleaned_Titles,Unnamed: 0,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,level_0,0
0,NI BIEN NI MAL,0,Bad Bunny,X 100PRE,NI BIEN NI MAL,278kSqsZIiYp8p3QjYAqa8,0.826,0.449,5,-8.330,0,0.0674,0.000018,0.2730,0.337,128.028,236216,4,0.0,Yeh-yeh\nYeh-yeh\nYeh-yeh\n\nSin ti no me va b...
1,200 Mph,1,Bad Bunny,X 100PRE,200 Mph,5jGUJXnELDaDTgiRjD9lsg,0.850,0.500,8,-6.708,1,0.0665,0.000384,0.0947,0.816,160.020,170509,4,1.0,¡Ju-Ju!\nYeh-yeah-yeah\nYeah-yeah-yeah-yeah\n\...
2,Cuando Perriabas,2,Bad Bunny,X 100PRE,Cuando Perriabas,5mj8WVFcKdGA8p9HWGTSLc,0.787,0.705,0,-7.582,1,0.0695,0.000001,0.1080,0.499,81.998,188654,4,2.0,Yeah-yeah-yeah\nYeah\nYo te llevo pa' Coachell...
3,La Romana,3,Bad Bunny,X 100PRE,La Romana,1khmgu0pveJbkbpbkyvcQv,0.655,0.725,0,-5.497,1,0.1880,0.002640,0.0611,0.326,125.800,300579,4,3.0,"(Yeh, yeh, yeh)\n\nPásame la hookah, ¡eh!\nQue..."
4,Como Antes,4,Bad Bunny,X 100PRE,Como Antes,69ZaPBHhRMRDjRpW1ivnOU,0.767,0.379,0,-10.348,1,0.0385,0.000145,0.2170,0.252,103.956,230578,4,4.0,"Yeh-yeh-yeh-yeh, ey\n\nAyer me acordé de la pr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,BTS Cypher 4,372,BTS,Wings,BTS Cypher 4,1QWxddJmOhQb1vDdyTipMR,0.817,0.797,1,-4.395,0,0.0840,0.000008,0.0551,0.362,119.977,294463,4,369.0,"이름, 이름 sorry bae\n발음, 발음 sorry bae\n딕션, 딕션, 딕션..."
373,Am I Wrong,373,BTS,Wings,Am I Wrong,1ORnZ5RmTKhVxPrjORKxAE,0.761,0.834,6,-4.992,1,0.0494,0.000010,0.0868,0.680,107.988,213082,4,370.0,"Ooh, ooh\nOoh, ooh\n\nAm I wrong for thinkin' ..."
374,21st Century Girl,374,BTS,Wings,21st Century Girl,6RioKpKPuG1vuyU0wnlAZx,0.716,0.906,2,-3.539,1,0.0717,0.000000,0.0547,0.762,101.028,192597,4,371.0,Give me an inch\nI promise I'll take a mile\nD...
375,2! 3!,375,BTS,Wings,2! 3!,0ma7lbs8BsVIv2kSaqOsDI,0.439,0.928,7,-4.096,1,0.2330,0.000000,0.1000,0.599,169.928,272537,4,372.0,"Yeah, yeah, yeah, yeah, yeah\nYeah, yeah, yeah..."


In [ ]:
#save as a csv
df_data.to_csv('/content/drive/MyDrive/QRML/projects/final/data/full_df', encoding='utf-8')